In [1]:
from quantrocket import get_prices
import pandas as pd
import numpy as np

In [2]:
# Step 1: Fetch daily close prices of Apple stock for the year 2023
start_date = '2023-01-01'
end_date = '2023-12-31'
prices = get_prices("usstock-free-1d", universes="usstock-free", start_date=start_date, end_date=end_date, fields=["Close"])
close_prices = prices['FIBBG000B9XRY4']

print('\n',close_prices)


 Field  Date      
Close  2023-01-03    124.2163
       2023-01-04    125.4975
       2023-01-05    124.1666
       2023-01-06    128.7352
       2023-01-09    129.2616
                       ...   
       2023-12-22    193.3533
       2023-12-26    192.8040
       2023-12-27    192.9038
       2023-12-28    193.3333
       2023-12-29    192.2846
Name: FIBBG000B9XRY4, Length: 250, dtype: float64


In [3]:
dates = close_prices.index.get_level_values('Date').date.tolist()
dates = [date.strftime('%Y-%m-%d') for date in dates]
print(dates)

['2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06', '2023-01-09', '2023-01-10', '2023-01-11', '2023-01-12', '2023-01-13', '2023-01-17', '2023-01-18', '2023-01-19', '2023-01-20', '2023-01-23', '2023-01-24', '2023-01-25', '2023-01-26', '2023-01-27', '2023-01-30', '2023-01-31', '2023-02-01', '2023-02-02', '2023-02-03', '2023-02-06', '2023-02-07', '2023-02-08', '2023-02-09', '2023-02-10', '2023-02-13', '2023-02-14', '2023-02-15', '2023-02-16', '2023-02-17', '2023-02-21', '2023-02-22', '2023-02-23', '2023-02-24', '2023-02-27', '2023-02-28', '2023-03-01', '2023-03-02', '2023-03-03', '2023-03-06', '2023-03-07', '2023-03-08', '2023-03-09', '2023-03-10', '2023-03-13', '2023-03-14', '2023-03-15', '2023-03-16', '2023-03-17', '2023-03-20', '2023-03-21', '2023-03-22', '2023-03-23', '2023-03-24', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30', '2023-03-31', '2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06', '2023-04-10', '2023-04-11', '2023-04-12', '2023-04-13', '2023-04-14', '2023

In [4]:
# Step 2: Calculate daily returns and classify states
returns = close_prices.pct_change()
print('Returns',returns)
states = np.where(returns >= 0.01, 1, np.where(returns > -0.01, 0, -1))
print('States',states)

Returns Field  Date      
Close  2023-01-03         NaN
       2023-01-04    0.010314
       2023-01-05   -0.010605
       2023-01-06    0.036794
       2023-01-09    0.004089
                       ...   
       2023-12-22   -0.005547
       2023-12-26   -0.002841
       2023-12-27    0.000518
       2023-12-28    0.002226
       2023-12-29   -0.005424
Name: FIBBG000B9XRY4, Length: 250, dtype: float64
States [-1  1 -1  1  0  0  1  0  1  0  0  0  1  1  1  0  1  1 -1  0  0  1  1 -1
  1 -1  0  0  1  0  1 -1  0 -1  0  0 -1  0  0 -1  0  1  1 -1  0 -1 -1  1
  1  0  1  0  1  1  0  0  0 -1  0  1  0  1  0  0 -1  0 -1  0  0  1  0  0
  0  0  0  0  0  0  0  1  0  0  0  0  0  1  0  0  1  0  0  0  0  0  1  0
  0 -1  0  0  1  1  0  1  0  0  0  0  1  0  1  0  0  1  0  0  0  1  0  0
  1  0  0  1  0  0  0  0 -1  0  0  0  0  1  0  0 -1  0  0  0  0  0  1  0
  0 -1  0 -1 -1  0  0  0  0  0 -1  0 -1  0  0  0  1 -1  1  0  1  1  0  0
  0 -1 -1  0  0 -1 -1  0  0  1  0 -1  0  0  0 -1  0  0  0  1  0  0  0  1
  0

In [6]:
# Step 3: Implement the value function
portfolio_value = 0
state_transitions = np.zeros((3, 3))
optimal_buy_indices = []

for i in range(1, len(states)):
    transition_distribution = state_transitions / (np.sum(state_transitions, axis=1)[:, None] + 1e-9)
    if states[i] == 1 and states[i-1] == 0 and transition_distribution[1, 2] > transition_distribution[1, 0]:
        portfolio_value += 1
        optimal_buy_indices.append(i-1)
        state_transitions[states[i-1] + 1, states[i] + 1] += 1
    elif states[i] == -1 and states[i-1] == 0 and transition_distribution[0, 1] > transition_distribution[0, 2]:
        portfolio_value -= 1
        state_transitions[states[i-1] + 1, states[i] + 1] += 1
    else:
        state_transitions[states[i-1] + 1, states[i] + 1] += 1
        continue

In [7]:
# Step 4: Calculate transition distribution
transition_distribution = state_transitions / np.sum(state_transitions, axis=1)[:, None]
print(transition_distribution)

[[0.13888889 0.72222222 0.13888889]
 [0.14649682 0.59872611 0.25477707]
 [0.125      0.67857143 0.19642857]]


In [11]:
print("Portfolio Value:", portfolio_value)
print("Optimal Index:",optimal_buy_indices)
print("\nTransition Distribution:")
print(pd.DataFrame(transition_distribution, index=['Bear', 'Flat', 'Bull'], columns=['Bear', 'Flat', 'Bull']))

Portfolio Value: 17
Optimal Index: [7, 11, 15, 20, 27, 29, 40, 49, 51, 58, 60, 68, 78, 84, 87, 93, 99, 102, 107, 109, 112, 116, 119, 122, 132, 141, 159, 163, 176, 186, 190, 206, 208, 211, 215, 217, 231, 233, 237]

Transition Distribution:
          Bear      Flat      Bull
Bear  0.138889  0.722222  0.138889
Flat  0.146497  0.598726  0.254777
Bull  0.125000  0.678571  0.196429


In [14]:
print('Corresponding Optimal Dates :')
for i in optimal_buy_indices:
    print(dates[i])

Corresponding Optimal Dates :
2023-01-12
2023-01-19
2023-01-25
2023-02-01
2023-02-10
2023-02-14
2023-03-02
2023-03-15
2023-03-17
2023-03-28
2023-03-30
2023-04-12
2023-04-26
2023-05-04
2023-05-09
2023-05-17
2023-05-25
2023-05-31
2023-06-07
2023-06-09
2023-06-14
2023-06-21
2023-06-26
2023-06-29
2023-07-14
2023-07-27
2023-08-22
2023-08-28
2023-09-15
2023-09-29
2023-10-05
2023-10-27
2023-10-31
2023-11-03
2023-11-09
2023-11-13
2023-12-04
2023-12-06
2023-12-12
